In [1]:
#!/usr/bin/env python3
# 53907afe-531b-11ea-a595-00505601122b
# b7ea974c-d389-11e8-a4be-00505601122b

# 1. Setup

## 1.1. FS/OS Requirements

In [2]:
!cp /kaggle/input/3d-recognition/modelnet.py /kaggle/working/modelnet.py
!cp /kaggle/input/3d-recognition/modelnet20.npz /kaggle/working/modelnet20.npz
!cp /kaggle/input/3d-recognition/modelnet32.npz /kaggle/working/modelnet32.npz

In [3]:
#!pip install -U tensorflow-gpu==2.8 tensorflow-addons==0.16.1 tensorflow-probability==0.16.0 tensorflow-hub==0.12.0 scipy
!pip freeze | grep tensorflow

## 1.2. Python imports

In [4]:
import argparse
import datetime
import os
import re

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#import fasttext
#import fasttext.util

from modelnet import ModelNet

## 1.3. Embeddings?

## 1.4. Args

In [17]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=None, type=int, help="Batch size.")
parser.add_argument("--epochs", default=None, type=int, help="Number of epochs.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=1, type=int, help="Maximum number of threads to use.")
parser.add_argument("--modelnet", default=20, type=int, help="ModelNet dimension.")
parser.add_argument("--checkpoints_period", default=None, type=int, help="Checkpoint callback period.")
parser.add_argument("--stopping_patience", default=None, type=int, help="Early stopping epochs patience.")
parser.add_argument("--label_smoothing", default=None, type=float, help="")
parser.add_argument("--densenet_filters", default=32, type=int, help="")
parser.add_argument("--densenet_block_sizes", nargs="+", type=int, default=[6, 12, 24, 16], help="Individual dense block sizes")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial model learning rate.")

args = parser.parse_args([
    '--batch_size=32',
    '--epochs=10',
    '--checkpoints_period=3',
    '--stopping_patience=3',
    '--learning_rate=0.0005',
    '--modelnet=32',
    '--label_smoothing=0.1',
    '--densenet_filters=32',
    '--densenet_block_sizes', '3', '6', '10', '8',
] if "__file__" not in globals() else None)

# Create logdir name
args.logdir = os.path.join(
    "logs",
    "{}-{}-{}".format(
        os.path.basename(globals().get("__file__", "notebook")),
        datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"),
        ",".join(
            (
                "{}={}".format(re.sub("(.)[^_]*_?", r"\1", k), v)
                for k, v in sorted(vars(args).items())
            )
        ),
    ),
)

tf.random.set_seed(args.seed) # tf2.6 (I have gpu issues on tf2.8 unfortunately)
tf.config.threading.set_inter_op_parallelism_threads(args.threads)
tf.config.threading.set_intra_op_parallelism_threads(args.threads)

args

In [18]:
args.decay_steps = int(args.epochs * modelnet.train.size / args.batch_size)

# 2. Data

In [7]:
modelnet = ModelNet(args.modelnet)
args.decay_steps = int(args.epochs * modelnet.train.size / args.batch_size)
print(modelnet.LABELS)

In [8]:
plt.figure(figsize=(14,7))
sns.histplot(modelnet.train.data['labels'], discrete=True, kde=True)

In [9]:
plt.figure(figsize=(14,7))
sns.histplot(modelnet.dev.data['labels'], discrete=True, kde=True)

In [19]:
def create_dataset(name):
    def prepare_example(example):
        return (
            example["voxels"],
            example["labels"] if args.label_smoothing is None else tf.one_hot(example["labels"], len(modelnet.LABELS))
        )

    dataset = getattr(modelnet, name).dataset.map(prepare_example)
    dataset = dataset.shuffle(len(dataset), seed=args.seed) if name == "train" else dataset
    #dataset = dataset.apply(tf.data.experimental.dense_to_ragged_batch(args.batch_size))
    dataset = dataset.batch(args.batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train, dev, test = create_dataset("train"), create_dataset("dev"), create_dataset("test")

# 3. Model

In [11]:
def _dense_block_part(hidden, filters, kernel, activation="relu"):
    hidden = tf.keras.layers.BatchNormalization()(hidden)
    hidden = tf.keras.layers.Activation(activation)(hidden)
    return tf.keras.layers.Conv3D(filters, kernel, 1, padding="same")(hidden)

def dense_block(hidden, filters, dense_block_size):
    for _ in range(dense_block_size):
        hidden_part = _dense_block_part(hidden, 4 * filters, 1) # 1x1 kernel conv layer
        hidden_part = _dense_block_part(hidden_part, filters, 3) # 3x3 kernel conv layer
        hidden = tf.keras.layers.Concatenate()([hidden_part, hidden]) # append output to residualy connected hidden inputs
    return hidden

def transition_layer(hidden):
    hidden = _dense_block_part(hidden, hidden.shape[-1] // 2, 1)
    return tf.keras.layers.MaxPool3D(2, 2, padding="same")(hidden)
    

# Architecture inspired from DenseNet121 (https://arxiv.org/pdf/1608.06993.pdf)
# mostly just reduced parameters and other specifics
# 
# Idea of _dense_block_part doing BN -> Activ -> Conv instead of Conv -> BN -> Activ
# is taken from https://towardsdatascience.com/creating-densenet-121-with-tensorflow-edbc08a956d8
def build_denselike_net(filters, dense_block_sizes):    
    inputs = tf.keras.layers.Input(shape=[modelnet.D, modelnet.H, modelnet.W, modelnet.C], dtype=tf.float32)
    hidden = tf.keras.layers.Conv3D(64, 7, 2, "same")(inputs)
    hidden = tf.keras.layers.MaxPooling3D(3, 2)(hidden)
    
    for dense_block_size in dense_block_sizes:
        hidden_part = dense_block(hidden, filters, dense_block_size)
        hidden = transition_layer(hidden_part)
    
    hidden = tf.keras.layers.GlobalAveragePooling3D()(hidden_part)
    #hidden = tf.keras.layers.Flatten()(hidden)
    #hidden = tf.keras.layers.Dense(128)(hidden)
    #hidden = tf.keras.layers.BatchNormalization()(hidden)
    #hidden = tf.keras.layers.Activation('relu')(hidden)
    outputs = tf.keras.layers.Dense(len(modelnet.LABELS), activation="softmax")(hidden)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

model = build_denselike_net(args.densenet_filters, args.densenet_block_sizes)

if args.label_smoothing:
    loss = tf.losses.CategoricalCrossentropy(label_smoothing=args.label_smoothing)
    metrics = [tf.metrics.CategoricalAccuracy(name="accuracy")]
else:
    loss = tf.losses.SparseCategoricalCrossentropy(label_smoothing=args.label_smoothing)
    metrics = [tf.metrics.SparseCategoricalAccuracy(name="accuracy")]

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=tf.keras.optimizers.schedules.CosineDecay(args.learning_rate, args.decay_steps)),
    loss=loss,
    metrics=metrics,
)
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model)

In [20]:
logs = model.fit(
    train, 
    epochs=args.epochs,
    validation_data=dev,
    shuffle=False,
    callbacks=[],
)

# 4. Predictions

In [21]:
os.makedirs(args.logdir, exist_ok=True)
with open(
    os.path.join(args.logdir, "3d_recognition.txt"), "w", encoding="utf-8"
) as predictions_file:
    # TODO: Predict the probabilities on the test set
    test_probabilities = model.predict(test)

    for probs in test_probabilities:
        print(np.argmax(probs), file=predictions_file)

In [ ]:
args.logdir